# Create the adjacency matrix based on the network distance

In [ ]:
import pandas as pd
import geopandas as gpd
import igraph as ig
import numpy as np
from shapely.geometry import Point, LineString, MultiLineString
from scipy.sparse import lil_matrix

In [ ]:
# Load the shapefile
shapefile_path = '/content/drive/MyDrive/traffic predict/traffic_flow_data/20211110_glasgow_road_link.shp'

# Load the shapefile and check for multi-part geometries
road = gpd.read_file(shapefile_path)
road = road[['TOID','startNode','endNode','SHAPE_Leng','geometry']]

In [ ]:
sensor_locations = pd.read_csv('/content/drive/MyDrive/traffic predict/traffic_flow_data/locations.csv')

In [ ]:
print(road.loc[794].geometry)
print(road.loc[793].geometry)
len(road)

MULTILINESTRING Z ((251248.17809751772 669639.0000000012 6.399999999994179, 251247.0000000005 669639.0000000012 6.399999999994179, 251245.95858065385 669639.2603548367 6.399999999994179), (251238.56264667233 669639.112529336 6.399999999994179, 251238.00000000026 669639.0000000012 6.399999999994179, 251230.00000000055 669635.3069999991 6.5, 251179.45799999978 669611.9780000006 6.69999999999709, 251174.4590000003 669607.2569999994 6.69999999999709, 251170.01599999957 669602.3509999997 6.69999999999709), (251256.00000000073 669631.9999999998 6.399999999994179, 251254.49207804582 669634.0105626032 6.399999999994179))
LINESTRING Z (266867.00000000035 664490.0000000007 60.10000000000582, 266914.00000000023 664422.9999999993 59.69999999999709, 266922 664409.9999999993 59.80000000000291)


32179

In [ ]:
# Reconstruct the road geodataframe to make sure each row as a single edge of the road network
# split a LineString or MultiLineString into two-point LineStrings
def split_to_edges(geometry):
    """Splits a LineString or MultiLineString geometry into individual two-point LineStrings."""
    edges = []

    if isinstance(geometry, LineString):
        # Split LineString into two-point segments
        points = list(geometry.coords)
        for i in range(len(points) - 1):
            edges.append(LineString([points[i], points[i+1]]))

    elif isinstance(geometry, MultiLineString):
        # First break MultiLineString into individual LineStrings
        for line in geometry.geoms:
            # Then split each LineString into two-point segments
            points = list(line.coords)
            for i in range(len(points) - 1):
                edges.append(LineString([points[i], points[i+1]]))

    return edges

# Create a new list to store the two-point LineStrings
new_geometries = []

# Iterate over each row in the GeoDataFrame and split the geometries
for geom in road.geometry:
    new_geometries.extend(split_to_edges(geom))

# Create a new GeoDataFrame from the split geometries
all_edges = gpd.GeoDataFrame(geometry=new_geometries, crs=road.crs)

# Display the first few rows of the new GeoDataFrame
all_edges.head(10)


,geometry
0,"LINESTRING Z (265498 666414 91.5, 265498 66641..."
1,"LINESTRING Z (265498 666416 91.5, 265491.164 6..."
2,"LINESTRING Z (252430 668308 16.1, 252401 66825..."
3,"LINESTRING Z (252401 668254 15.5, 252398.6 668..."
4,"LINESTRING Z (252398.6 668250 15.4, 252389 668..."
5,"LINESTRING Z (252389 668234 14.5, 252369 66820..."
6,"LINESTRING Z (252369 668208 12.8, 252348 66817..."
7,"LINESTRING Z (252348 668177 10.9, 252306 66812..."
8,"LINESTRING Z (252066.564 660968.5 27.9, 252075..."
9,"LINESTRING Z (252075.98 660958.817 28.1, 25208..."


In [ ]:
len(all_edges)

143751

In [ ]:
# Step 1: Convert sensor locations to GeoDataFrame with Point geometries
sensor_locations['geometry'] = sensor_locations.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
sensor_gdf = gpd.GeoDataFrame(sensor_locations, geometry='geometry', crs='EPSG:4326')  # Assuming WGS84 coordinate system

# Convert road edges and sensor locations to the same projection if necessary (road_edges may use a different CRS)
sensor_gdf = sensor_gdf.to_crs(road.crs)

In [ ]:
# Step 2: Calculate the perpendicular distance from each sensor to each edge in the road network
# We'll loop through each sensor and find the nearest edge by minimizing the perpendicular distance

nearest_edges = []  # This will store the nearest edge for each sensor

# Loop through each sensor and find the nearest edge by perpendicular distance
for sensor in sensor_gdf.geometry:
    distances = all_edges.geometry.apply(lambda edge: sensor.distance(edge))  # Perpendicular distance to each edge
    nearest_edge_idx = np.argmin(distances)  # Index of the nearest edge
    nearest_edges.append(all_edges.iloc[nearest_edge_idx])  # Store the nearest edge

# Create a GeoDataFrame with the nearest edges corresponding to each sensor
sensor_to_edge_mapping = gpd.GeoDataFrame(nearest_edges, geometry='geometry')

# Step 3: Filter the unique edges that have sensors mapped to them
sensor_edges = sensor_to_edge_mapping.drop_duplicates()


In [ ]:
len(sensor_edges)

444

In [ ]:
# Step 4: Prepare for calculating road network distances between the mapped sensor edges
# Create a list of all vertices (start and end points of the edges)
vertices = {}  # To map points (start, end) to unique IDs
vertex_id = 0

edges = []  # To store the edges as (start_vertex, end_vertex, distance)
for idx, row in all_edges.iterrows():
    line = row.geometry
    start_point = Point(line.coords[0])
    end_point = Point(line.coords[-1])

    if start_point not in vertices:
        vertices[start_point] = vertex_id
        vertex_id += 1
    if end_point not in vertices:
        vertices[end_point] = vertex_id
        vertex_id += 1

    # Add the edge (start vertex, end vertex, length of the road segment)
    edge_length = start_point.distance(end_point)  # Assuming Euclidean distance for the edge length
    edges.append((vertices[start_point], vertices[end_point], edge_length))

In [ ]:
# Step 5: Create an igraph Graph object for the full road network
g = ig.Graph()
g.add_vertices(len(vertices))  # Add vertices to the graph (number of unique points in the network)
g.add_edges([(e[0], e[1]) for e in edges])  # Add the edges (start_vertex, end_vertex) to the graph
g.es['weight'] = [e[2] for e in edges]  # Set the edge weights as road segment lengths (distances)

In [ ]:
# Step 6: Function to calculate the shortest path between the start vertices of two sensor edges
def get_dynamic_network_distance(edge1, edge2):
    start_v1 = edge1['geometry'].coords[0]  # Start vertex of edge 1
    start_v2 = edge2['geometry'].coords[0]  # Start vertex of edge 2

    # Retrieve the corresponding vertex IDs from the vertices dictionary
    vertex_id_1 = vertices[Point(start_v1)]
    vertex_id_2 = vertices[Point(start_v2)]

    # Use Dijkstra's algorithm to compute the shortest path between the two vertices dynamically
    distance = g.distances(source=vertex_id_1, target=vertex_id_2, weights='weight')[0][0]

    return distance

In [ ]:
# Step 7: Compute the road network distances between each pair of sensor edges dynamically
num_sensors = len(sensor_edges)
sensor_adjacency_matrix = lil_matrix((num_sensors, num_sensors), dtype=np.float32)

for i in range(num_sensors):
    for j in range(i + 1, num_sensors):  # Only compute for upper triangle to avoid redundant calculations
        print(i,j)
        edge1 = sensor_edges.iloc[i]
        edge2 = sensor_edges.iloc[j]

        # Get the shortest road network distance between the start vertices of the sensor edges
        distance = get_dynamic_network_distance(edge1, edge2)

        # Fill the adjacency matrix symmetrically
        sensor_adjacency_matrix[i, j] = distance
        sensor_adjacency_matrix[j, i] = distance  # Symmetry since the graph is undirected

# Now sensor_adjacency_matrix contains the road network distances between all sensor edges

Streaming output truncated to the last 5000 lines.
343 394
343 395
343 396
343 397
343 398
343 399
343 400
343 401
343 402
343 403
343 404
343 405
343 406
343 407
343 408
343 409
343 410
343 411
343 412
343 413
343 414
343 415
343 416
343 417
343 418
343 419
343 420
343 421
343 422
343 423
343 424
343 425
343 426
343 427
343 428
343 429
343 430
343 431
343 432
343 433
343 434
343 435
343 436
343 437
343 438
343 439
343 440
343 441
343 442
343 443
344 345
344 346
344 347
344 348
344 349
344 350
344 351
344 352
344 353
344 354
344 355
344 356
344 357
344 358
344 359
344 360
344 361
344 362
344 363
344 364
344 365
344 366
344 367
344 368
344 369
344 370
344 371
344 372
344 373
344 374
344 375
344 376
344 377
344 378
344 379
344 380
344 381
344 382
344 383
344 384
344 385
344 386
344 387
344 388
344 389
344 390
344 391
344 392
344 393
344 394
344 395
344 396
344 397
344 398
344 399
344 400
344 401
344 402
344 403
344 404
344 405
344 406
344 407
344 408
344 409
344 410
344 411
344 412
344 4

In [ ]:
dense_matrix = sensor_adjacency_matrix.toarray()

In [ ]:
# Save the dense matrix as a CSV file
np.savetxt('/content/drive/MyDrive/traffic predict/dense_matrix.csv', dense_matrix, delimiter=',')

# **Data pre-proess**

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import random
from itertools import product
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

2024-10-12 15:54:51.544976: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-12 15:54:52.342109: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-12 15:54:52.606166: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-12 15:54:52.687196: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-12 15:54:53.265497: I tensorflow/core/platform/cpu_feature_guar

Sensors:

GD030A_S, GD025A_B, GD0451_S

In [2]:
traffic_data = pd.read_csv('GD030A_S.csv')

## 1. Recover timestamp

In [3]:
# Define the recover_timestamp function
def recover_timestamp(data):
    # Combine 'date' and 'time' to form a datetime column
    data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['time'].astype(str) + ':00', format='%Y-%m-%d %H:%M')

    # Set 'datetime' as index
    data = data.set_index('datetime')

    # Create a complete range of timestamps with hourly frequency
    full_time_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='H')

    # Reindex the data to include all timestamps, filling missing rows with NaN
    data_full = data.reindex(full_time_range)

    return data_full

In [4]:
# Apply the recover_timestamp function to recover the full time series
traffic_full = recover_timestamp(traffic_data)
traffic_full

/tmp/ipykernel_636502/36863830.py:10: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_time_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='H')


,date,time,flow
2019-10-01 00:00:00,2019-10-01,0.0,15.0
2019-10-01 01:00:00,2019-10-01,1.0,9.0
2019-10-01 02:00:00,2019-10-01,2.0,9.0
2019-10-01 03:00:00,2019-10-01,3.0,7.0
2019-10-01 04:00:00,2019-10-01,4.0,9.0
...,...,...,...
2023-09-30 19:00:00,2023-09-30,19.0,129.0
2023-09-30 20:00:00,2023-09-30,20.0,119.0
2023-09-30 21:00:00,2023-09-30,21.0,106.0
2023-09-30 22:00:00,2023-09-30,22.0,88.0


In [5]:
import plotly.graph_objects as go
# Create interactive plot using Plotly
fig = go.Figure()

# Add observed data to the plot
fig.add_trace(go.Scatter(x=traffic_full['flow'].index, y=traffic_full['flow'], mode='lines', name='Observed Data'))

# Update layout for better visualization
fig.update_layout(
    title='Observed Data ',
    xaxis_title='Date',
    yaxis_title='Traffic Flow',
    legend_title='Legend'
)

# Show the plot
fig.show()

ModuleNotFoundError: No module named 'plotly'

## 2. Train, validate, test data split

In [11]:
# train_set = traffic_full[:'2022-02-28 23:00:00']
# valid_set = traffic_full['2022-03-01 00:00:00':'2022-12-31 23:00:00']
# test_set = traffic_full['2023-01-01 00:00:00':]
train_set = traffic_full['2022-06-03 00:00:00':'2023-03-31 23:00:00']
valid_set = traffic_full['2023-04-01 00:00:00':'2023-06-30 23:00:00']
test_set = traffic_full['2023-07-01 00:00:00':]
print('Proportion of train_set : {:.4f}'.format(len(train_set)/len(traffic_full['2022-06-03 00:00:00':])))
print('Proportion of valid_set : {:.4f}'.format(len(valid_set)/len(traffic_full['2022-06-03 00:00:00':])))
print('Proportion of test_set : {:.4f}'.format(len(test_set)/len(traffic_full['2022-06-03 00:00:00':])))

Proportion of train_set : 0.6227
Proportion of valid_set : 0.1876
Proportion of test_set : 0.1897


In [12]:
print(train_set.isnull().sum(), len(train_set))
print(valid_set.isnull().sum(),len(valid_set))
print(test_set.isnull().sum(),len(test_set))

date    16
time    16
flow    16
dtype: int64 7248
date    61
time    61
flow    61
dtype: int64 2184
date    342
time    342
flow    342
dtype: int64 2208


## 3. Normalise the data 

In [13]:
# Initialize the scaler
scaler = MinMaxScaler()

# Fit the scaler on the training data's 'flow' feature
scaler.fit(train_set[['flow']])

# Transform the 'flow' feature in all datasets
train_set.loc[:, 'flow_scaled'] = scaler.transform(train_set[['flow']])
valid_set.loc[:, 'flow_scaled'] = scaler.transform(valid_set[['flow']])
test_set.loc[:, 'flow_scaled'] = scaler.transform(test_set[['flow']])

/tmp/ipykernel_636502/1100395100.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set.loc[:, 'flow_scaled'] = scaler.transform(train_set[['flow']])
/tmp/ipykernel_636502/1100395100.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_set.loc[:, 'flow_scaled'] = scaler.transform(valid_set[['flow']])
/tmp/ipykernel_636502/1100395100.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

## 4. Split the data into X and y

In [14]:
def create_sequences(data, input_length, forecast_horizon):
    """
    Creates input-output sequences for time series data, excluding any sequences containing NaN values.
    
    Parameters:
    - data: pandas DataFrame containing the data. Must include the 'flow_scaled' column.
    - input_length: int, number of past time steps to include in each input sequence.
    - forecast_horizon: int, number of future steps to predict.
    
    Returns:
    - X: numpy array of shape (num_valid_samples, input_length, num_features)
    - y: numpy array of shape (num_valid_samples, forecast_horizon)
    """
    X, y = [], []
    num_features = data.shape[1]
    total_length = input_length + forecast_horizon
    
    for i in range(input_length, len(data) - forecast_horizon + 1):
        # Extract the input sequence
        X_seq = data.iloc[i - input_length:i]['flow_scaled'].values
        # Extract the target sequence
        y_seq = data.iloc[i:i + forecast_horizon]['flow_scaled'].values
        
        # Check for NaN values in the input sequence and target sequence
        if not np.isnan(X_seq).any() and not np.isnan(y_seq).any():
            X.append(X_seq)
            y.append(y_seq)
        else:
            # Optionally, log or count the skipped sequences
            pass  # Simply skip sequences with NaNs
        
    # Convert to numpy arrays and reshape X to match LSTM expected input (samples, timesteps, features)
    X = np.array(X).reshape(-1, input_length, 1)
    y = np.array(y).reshape(-1, forecast_horizon)
    
    return X, y

## 5. Create X and y

#### We will use
* the last 24*N steps

*  to forecast current (0 step) and 5 steps ahead

In [15]:
# Define Input Sequence Lengths
input_lengths = [24 * i for i in range(1, 22)]  # [24, 48, ..., 168]

In [16]:
from collections import defaultdict
data_dict = defaultdict(dict)

for length in input_lengths:
    print(f"Processing input length: {length}")
    
    # Create sequences with forecast_horizon=6
    X_train, y_train = create_sequences(train_set, length, forecast_horizon=6)
    X_val, y_val = create_sequences(valid_set, length, forecast_horizon=6)
    X_test, y_test = create_sequences(test_set, length, forecast_horizon=6)
    
    # Store in the dictionary
    data_dict[length]['X_train'] = X_train
    data_dict[length]['y_train'] = y_train
    data_dict[length]['X_val'] = X_val
    data_dict[length]['y_val'] = y_val
    data_dict[length]['X_test'] = X_test
    data_dict[length]['y_test'] = y_test
    
    # Print shapes and ensure no NaNs
    print(f"  X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
    print(f"  X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
    print(f"  X_test shape: {X_test.shape}, y_test shape: {y_test.shape}\n")

Processing input length: 408
  X_train shape: (5993, 408, 1), y_train shape: (5993, 6)
  X_val shape: (603, 408, 1), y_val shape: (603, 6)
  X_test shape: (342, 408, 1), y_test shape: (342, 6)

Processing input length: 432
  X_train shape: (5921, 432, 1), y_train shape: (5921, 6)
  X_val shape: (531, 432, 1), y_val shape: (531, 6)
  X_test shape: (311, 432, 1), y_test shape: (311, 6)

Processing input length: 456
  X_train shape: (5849, 456, 1), y_train shape: (5849, 6)
  X_val shape: (459, 456, 1), y_val shape: (459, 6)
  X_test shape: (287, 456, 1), y_test shape: (287, 6)

Processing input length: 480
  X_train shape: (5777, 480, 1), y_train shape: (5777, 6)
  X_val shape: (387, 480, 1), y_val shape: (387, 6)
  X_test shape: (263, 480, 1), y_test shape: (263, 6)

Processing input length: 504
  X_train shape: (5705, 504, 1), y_train shape: (5705, 6)
  X_val shape: (333, 504, 1), y_val shape: (333, 6)
  X_test shape: (239, 504, 1), y_test shape: (239, 6)



## 6. Buid the CNN model

In [17]:
def build_cnn_model(hyperparams, input_length):
    
    model = Sequential()

    # First Conv layer requires input shape
    model.add(layers.Conv1D(filters=hyperparams['filters'], 
                            kernel_size=hyperparams['kernel_size'],
                            activation='relu',
                            input_shape=(input_length, 1)))
    model.add(layers.Conv1D(filters=hyperparams['filters'], 
                            kernel_size=hyperparams['kernel_size'],
                            activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    
    model.add(layers.Flatten())
    model.add(layers.Dropout(rate=hyperparams['dropout']))
    model.add(layers.Dense(6))  # Output layer for multi-step forecasting
    
    # Compile the model with MSE as the loss function
    optimizer = keras.optimizers.Adam(learning_rate=hyperparams['learning_rate'])
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])
    
    return model

## 7. Define the hyperparameter grid

In [18]:
hyperparameter_grid = {               
    'filters': [32, 64, 128],                  # Number of filters in each Conv layer
    'kernel_size': [2, 3, 5],                     # Size of the convolutional kernels
    'dropout': [0.0, 0.1, 0.2, 0.3],                # Dropout rates to prevent overfitting
    'learning_rate': [0.001, 0.0005, 0.0001],          # Learning rates for the optimizer
    'batch_size': [32, 64, 128]                # Batch sizes for training
}

In [19]:
all_combinations = list(product(
    hyperparameter_grid['filters'],
    hyperparameter_grid['kernel_size'],
    hyperparameter_grid['dropout'],
    hyperparameter_grid['learning_rate'],
    hyperparameter_grid['batch_size']
))

In [20]:
len(all_combinations)

324

## 8. Train the model

In [21]:
results = []

for length in input_lengths:
    print(f"Starting grid search for input length: {length}")
    
    X_train = data_dict[length]['X_train']
    y_train = data_dict[length]['y_train']
    X_val = data_dict[length]['X_val']
    y_val = data_dict[length]['y_val']
    
    best_mse = float('inf')
    best_params = {}
    best_model = None
    
    for idx, combination in enumerate(all_combinations):
        # Extract hyperparameters
        hyperparams = {
            'filters': combination[0],
            'kernel_size': combination[1],
            'dropout': combination[2],
            'learning_rate': combination[3],
            'batch_size': combination[4]
        }
        
        print(f"  Evaluating combination {idx + 1}/{len(all_combinations)}: {hyperparams}")
        
        # Build the CNN model
        model = build_cnn_model(hyperparams, length)
        
        # Early Stopping Callback
        early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True,verbose=1)
        
        # Train the model
        history = model.fit(
            X_train, y_train,
            epochs=50,
            batch_size=hyperparams['batch_size'],
            validation_data=(X_val, y_val),
            callbacks=[early_stop],
            verbose=0  # Set to 1 to see training progress
        )
        
        # Retrieve the best validation MSE from the history
        current_best_mse = min(history.history['val_loss'])
        print(f"Validation loss: {current_best_mse:.5f}")
        
        # Check if this is the best model so far
        if current_best_mse < best_mse:
            best_mse = current_best_mse
            best_params = hyperparams.copy()
            best_model = model  # Optionally, save the model if needed
    
    # After evaluating all combinations, store the best results
    results.append({
        'Input_Length': length,
        'Best_MSE': best_mse,
        'Validation_MSE': mean_squared_error(y_val, best_model.predict(X_val)),
        'Validation_MAE': mean_absolute_error(y_val, best_model.predict(X_val)),
        'Validation_MAPE': mean_absolute_percentage_error(y_val, best_model.predict(X_val)) * 100,  # In percentage
        'Validation_RMSE': np.sqrt(mean_squared_error(y_val, best_model.predict(X_val))),
        'Best_Hyperparameters': best_params
    })
    
    print(f"Completed grid search for input length: {length}")
    print(f"  Best Validation MSE: {best_mse:.4f}")
    print(f"  Best Hyperparameters: {best_params}\n")

Starting grid search for input length: 408
  Evaluating combination 1/324: {'filters': 32, 'kernel_size': 2, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 32}


/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-10-12 15:56:48.447342: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Epoch 13: early stopping
Restoring model weights from the end of the best epoch: 3.
Validation loss: 0.00447
  Evaluating combination 2/324: {'filters': 32, 'kernel_size': 2, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 64}
Epoch 16: early stopping
Restoring model weights from the end of the best epoch: 6.
Validation loss: 0.00438
  Evaluating combination 3/324: {'filters': 32, 'kernel_size': 2, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 128}
Epoch 23: early stopping
Restoring model weights from the end of the best epoch: 13.
Validation loss: 0.00424
  Evaluating combination 4/324: {'filters': 32, 'kernel_size': 2, 'dropout': 0.0, 'learning_rate': 0.0005, 'batch_size': 32}
Epoch 16: early stopping
Restoring model weights from the end of the best epoch: 6.
Validation loss: 0.00438
  Evaluating combination 5/324: {'filters': 32, 'kernel_size': 2, 'dropout': 0.0, 'learning_rate': 0.0005, 'batch_size': 64}
Epoch 21: early stopping
Restoring model weights from the end of

/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 13: early stopping
Restoring model weights from the end of the best epoch: 3.
Validation loss: 0.00456
  Evaluating combination 2/324: {'filters': 32, 'kernel_size': 2, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 64}
Epoch 15: early stopping
Restoring model weights from the end of the best epoch: 5.
Validation loss: 0.00421
  Evaluating combination 3/324: {'filters': 32, 'kernel_size': 2, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 128}
Epoch 17: early stopping
Restoring model weights from the end of the best epoch: 7.
Validation loss: 0.00441
  Evaluating combination 4/324: {'filters': 32, 'kernel_size': 2, 'dropout': 0.0, 'learning_rate': 0.0005, 'batch_size': 32}
Epoch 16: early stopping
Restoring model weights from the end of the best epoch: 6.
Validation loss: 0.00436
  Evaluating combination 5/324: {'filters': 32, 'kernel_size': 2, 'dropout': 0.0, 'learning_rate': 0.0005, 'batch_size': 64}
Epoch 15: early stopping
Restoring model weights from the end of 

...

Epoch 13: early stopping
Restoring model weights from the end of the best epoch: 3.
Validation loss: 0.00486
  Evaluating combination 2/324: {'filters': 32, 'kernel_size': 2, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 64}
Epoch 13: early stopping
Restoring model weights from the end of the best epoch: 3.
Validation loss: 0.00480
  Evaluating combination 3/324: {'filters': 32, 'kernel_size': 2, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 128}
Epoch 18: early stopping
Restoring model weights from the end of the best epoch: 8.
Validation loss: 0.00464
  Evaluating combination 4/324: {'filters': 32, 'kernel_size': 2, 'dropout': 0.0, 'learning_rate': 0.0005, 'batch_size': 32}
Epoch 16: early stopping
Restoring model weights from the end of the best epoch: 6.
Validation loss: 0.00477
  Evaluating combination 5/324: {'filters': 32, 'kernel_size': 2, 'dropout': 0.0, 'learning_rate': 0.0005, 'batch_size': 64}
Epoch 18: early stopping
Restoring model weights from the end of 

/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 13: early stopping
Restoring model weights from the end of the best epoch: 3.
Validation loss: 0.00572
  Evaluating combination 2/324: {'filters': 32, 'kernel_size': 2, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 64}
Epoch 14: early stopping
Restoring model weights from the end of the best epoch: 4.
Validation loss: 0.00536
  Evaluating combination 3/324: {'filters': 32, 'kernel_size': 2, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 128}
Epoch 18: early stopping
Restoring model weights from the end of the best epoch: 8.
Validation loss: 0.00571
  Evaluating combination 4/324: {'filters': 32, 'kernel_size': 2, 'dropout': 0.0, 'learning_rate': 0.0005, 'batch_size': 32}
Epoch 18: early stopping
Restoring model weights from the end of the best epoch: 8.
Validation loss: 0.00557
  Evaluating combination 5/324: {'filters': 32, 'kernel_size': 2, 'dropout': 0.0, 'learning_rate': 0.0005, 'batch_size': 64}
Epoch 17: early stopping
Restoring model weights from the end of 

/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 13: early stopping
Restoring model weights from the end of the best epoch: 3.
Validation loss: 0.00645
  Evaluating combination 2/324: {'filters': 32, 'kernel_size': 2, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 64}
Epoch 14: early stopping
Restoring model weights from the end of the best epoch: 4.
Validation loss: 0.00573
  Evaluating combination 3/324: {'filters': 32, 'kernel_size': 2, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 128}
Epoch 16: early stopping
Restoring model weights from the end of the best epoch: 6.
Validation loss: 0.00549
  Evaluating combination 4/324: {'filters': 32, 'kernel_size': 2, 'dropout': 0.0, 'learning_rate': 0.0005, 'batch_size': 32}
Epoch 16: early stopping
Restoring model weights from the end of the best epoch: 6.
Validation loss: 0.00567
  Evaluating combination 5/324: {'filters': 32, 'kernel_size': 2, 'dropout': 0.0, 'learning_rate': 0.0005, 'batch_size': 64}
Epoch 15: early stopping
Restoring model weights from the end of 

Completed grid search for input length: 24
  Best Validation MSE: 0.0042
  Best Hyperparameters: {'filters': 128, 'kernel_size': 3, 'dropout': 0.3, 'learning_rate': 0.0005, 'batch_size': 64}

Completed grid search for input length: 48
  Best Validation MSE: 0.0042
  Best Hyperparameters: {'filters': 64, 'kernel_size': 3, 'dropout': 0.0, 'learning_rate': 0.0005, 'batch_size': 128}

Completed grid search for input length: 72
  Best Validation MSE: 0.0041
  Best Hyperparameters: {'filters': 64, 'kernel_size': 5, 'dropout': 0.3, 'learning_rate': 0.001, 'batch_size': 64}

Completed grid search for input length: 96
  Best Validation MSE: 0.0042
  Best Hyperparameters: {'filters': 64, 'kernel_size': 3, 'dropout': 0.0, 'learning_rate': 0.0005, 'batch_size': 128}

Completed grid search for input length: 120
  Best Validation MSE: 0.0040
  Best Hyperparameters: {'filters': 64, 'kernel_size': 3, 'dropout': 0.1, 'learning_rate': 0.0005, 'batch_size': 128}

Completed grid search for input length: 144
  Best Validation MSE: 0.0040
  Best Hyperparameters: {'filters': 32, 'kernel_size': 2, 'dropout': 0.3, 'learning_rate': 0.0005, 'batch_size': 32}

Completed grid search for input length: 168
  Best Validation MSE: 0.0038
  Best Hyperparameters: {'filters': 32, 'kernel_size': 2, 'dropout': 0.1, 'learning_rate': 0.0005, 'batch_size': 64}

Completed grid search for input length: 192
  Best Validation MSE: 0.0037
  Best Hyperparameters: {'filters': 128, 'kernel_size': 3, 'dropout': 0.2, 'learning_rate': 0.0001, 'batch_size': 32}

Completed grid search for input length: 216
  Best Validation MSE: 0.0034
  Best Hyperparameters: {'filters': 64, 'kernel_size': 3, 'dropout': 0.2, 'learning_rate': 0.0001, 'batch_size': 32}

Completed grid search for input length: 240
  Best Validation MSE: 0.0035
  Best Hyperparameters: {'filters': 32, 'kernel_size': 5, 'dropout': 0.0, 'learning_rate': 0.0001, 'batch_size': 32}
  
Completed grid search for input length: 264
  Best Validation MSE: 0.0036
  Best Hyperparameters: {'filters': 128, 'kernel_size': 3, 'dropout': 0.3, 'learning_rate': 0.0005, 'batch_size': 32}

Completed grid search for input length: 288
  Best Validation MSE: 0.0035
  Best Hyperparameters: {'filters': 32, 'kernel_size': 3, 'dropout': 0.3, 'learning_rate': 0.0001, 'batch_size': 32}

Completed grid search for input length: 312
  Best Validation MSE: 0.0035
  Best Hyperparameters: {'filters': 32, 'kernel_size': 2, 'dropout': 0.1, 'learning_rate': 0.0005, 'batch_size': 128}


Completed grid search for input length: 336
  Best Validation MSE: 0.0034
  Best Hyperparameters: {'filters': 64, 'kernel_size': 2, 'dropout': 0.2, 'learning_rate': 0.0001, 'batch_size': 32}

Completed grid search for input length: 360
  Best Validation MSE: 0.0035
  Best Hyperparameters: {'filters': 32, 'kernel_size': 2, 'dropout': 0.3, 'learning_rate': 0.0001, 'batch_size': 64}

Completed grid search for input length: 384
  Best Validation MSE: 0.0038
  Best Hyperparameters: {'filters': 32, 'kernel_size': 2, 'dropout': 0.1, 'learning_rate': 0.0001, 'batch_size': 128}

Completed grid search for input length: 408
  Best Validation MSE: 0.0040
  Best Hyperparameters: {'filters': 32, 'kernel_size': 2, 'dropout': 0.1, 'learning_rate': 0.001, 'batch_size': 128}

Completed grid search for input length: 432
  Best Validation MSE: 0.0039
  Best Hyperparameters: {'filters': 32, 'kernel_size': 3, 'dropout': 0.0, 'learning_rate': 0.0001, 'batch_size': 32}

Completed grid search for input length: 456
  Best Validation MSE: 0.0043
  Best Hyperparameters: {'filters': 32, 'kernel_size': 3, 'dropout': 0.0, 'learning_rate': 0.0001, 'batch_size': 64}

Completed grid search for input length: 480
  Best Validation MSE: 0.0048
  Best Hyperparameters: {'filters': 64, 'kernel_size': 2, 'dropout': 0.2, 'learning_rate': 0.0001, 'batch_size': 32}

Completed grid search for input length: 504
  Best Validation MSE: 0.0051
  Best Hyperparameters: {'filters': 32, 'kernel_size': 2, 'dropout': 0.1, 'learning_rate': 0.0001, 'batch_size': 64}

## 9. Retrain the model after getting the best hyperparameters of each input length

In [13]:
# Define the best hyperparameters for each input length

best_hyperparameters = {
    24: {
        'filters': 128,
        'kernel_size': 3,
        'dropout': 0.3,
        'learning_rate': 0.0005,
        'batch_size': 64
    },
    48: {
        'filters': 64,
        'kernel_size': 3,
        'dropout': 0.0,
        'learning_rate': 0.0005,
        'batch_size': 128
    },
    72: {
        'filters': 64,
        'kernel_size': 5,
        'dropout': 0.3,
        'learning_rate': 0.001,
        'batch_size': 64
    },
    96: {
        'filters': 64,
        'kernel_size': 3,
        'dropout': 0.0,
        'learning_rate': 0.0005,
        'batch_size': 128
    },
    120: {
        'filters': 64,
        'kernel_size': 3,
        'dropout': 0.1,
        'learning_rate': 0.0005,
        'batch_size': 128
    },
    144: {
        'filters': 32,
        'kernel_size': 2,
        'dropout': 0.3,
        'learning_rate': 0.0005,
        'batch_size': 32
    },
    168: {
        'filters': 32,
        'kernel_size': 2,
        'dropout': 0.1,
        'learning_rate': 0.0005,
        'batch_size': 64
    },
    192: {
        'filters': 128,
        'kernel_size': 3,
        'dropout': 0.2,
        'learning_rate': 0.0001,
        'batch_size': 32
    },
    216: {
        'filters': 64,
        'kernel_size': 3,
        'dropout': 0.2,
        'learning_rate': 0.0001,
        'batch_size': 32
    },
    240: {
        'filters': 32,
        'kernel_size': 5,
        'dropout': 0.0,
        'learning_rate': 0.0001,
        'batch_size': 32
    },
    264: {
        'filters': 128,
        'kernel_size': 3,
        'dropout': 0.3,
        'learning_rate': 0.0005,
        'batch_size': 32
    },
    288: {
        'filters': 32,
        'kernel_size': 3,
        'dropout': 0.3,
        'learning_rate': 0.0001,
        'batch_size': 32
    },
    312: {
        'filters': 32,
        'kernel_size': 2,
        'dropout': 0.1,
        'learning_rate': 0.0005,
        'batch_size': 128
    },
    336: {
        'filters': 64,
        'kernel_size': 2,
        'dropout': 0.2,
        'learning_rate': 0.0001,
        'batch_size': 32
    },
    360: {
        'filters': 32,
        'kernel_size': 2,
        'dropout': 0.3,
        'learning_rate': 0.0001,
        'batch_size': 64
    },
    384: {
        'filters': 32,
        'kernel_size': 2,
        'dropout': 0.1,
        'learning_rate': 0.0001,
        'batch_size': 128
    },
    408: {
        'filters': 32,
        'kernel_size': 2,
        'dropout': 0.1,
        'learning_rate': 0.001,
        'batch_size': 128
    },
    432: {
        'filters': 32,
        'kernel_size': 3,
        'dropout': 0.0,
        'learning_rate': 0.0001,
        'batch_size': 32
    },
    456: {
        'filters': 32,
        'kernel_size': 3,
        'dropout': 0.0,
        'learning_rate': 0.0001,
        'batch_size': 64
    },
    480: {
        'filters': 64,
        'kernel_size': 2,
        'dropout': 0.2,
        'learning_rate': 0.0001,
        'batch_size': 32
    },
    504: {
        'filters': 32,
        'kernel_size': 2,
        'dropout': 0.1,
        'learning_rate': 0.0001,
        'batch_size': 64
    }
}

In [14]:
def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)

In [15]:
def train_model(hyperparams,data_dict,length, seed=None):  # add seed
    if seed is not None:
        set_seed(seed)
    
    #get the data of each length
    X_train = data_dict[length]['X_train']
    y_train = data_dict[length]['y_train']
    X_val = data_dict[length]['X_val']
    y_val = data_dict[length]['y_val']
    
    # Train the model
    model = build_cnn_model(hyperparams, length)    
    # Early Stopping Callback
    early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True,verbose=1)    
    
    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=hyperparams['batch_size'],
        validation_data=(X_val, y_val),
        callbacks=[early_stop],
        verbose=0  # Set to 1 to see training progress
    )
    
    # Retrieve the best validation MSE from the history
    best_mse = min(history.history['val_loss'])
    print(f"Validation loss: {best_mse:.5f}")
    
    return model, best_mse

In [16]:
# Make predictions
def make_prediction(model, X_obs, y_obs):
    y_pred = model.predict(X_obs,verbose=0)
    n_samples = X_obs.shape[0]
    output_len = y_obs.shape[1]

    # Reshape for inverse scaling
    y_pred_reshaped = y_pred.reshape(-1, 1)
    y_obs_reshaped = y_obs.reshape(-1, 1)

    # Inverse transform
    y_pred_inverse = scaler.inverse_transform(y_pred_reshaped).reshape(n_samples, output_len)
    y_obs_inverse = scaler.inverse_transform(y_obs_reshaped).reshape(n_samples, output_len)

    return y_pred_inverse, y_obs_inverse

In [17]:
# Compute Metrics for Each Time Step
def evaluation(y_pred_inverse, y_obs_inverse):
    
    output_len = y_pred_inverse.shape[1]
    metrics_list = []  # To store metrics for each time step
    
    for i in range(output_len):
        y_true = y_obs_inverse[:, i]
        y_pred = y_pred_inverse[:, i]

        # Mean Absolute Error (MAE)
        mae = mean_absolute_error(y_true, y_pred)

        # Mean Squared Error (MSE)
        mse = mean_squared_error(y_true, y_pred)

        # Root Mean Squared Error (RMSE)
        rmse = np.sqrt(mse)

        # Mean Absolute Percentage Error (MAPE)
        # Avoid division by zero by adding a small epsilon to y_test_flat if necessary
        epsilon = 1e-10
        y_true_safe = np.where(y_true == 0, epsilon, y_true)
        mape = np.mean(np.abs((y_true - y_pred) / y_true_safe)) * 100

                # Append the metrics for the current time step to the list
        metrics_list.append({
            'Time Step': i + 1,
            'MAE': mae,
            'RMSE': rmse,
            'MAPE (%)': mape
        })

    # Create a DataFrame from the list of metrics
    metrics_df = pd.DataFrame(metrics_list)
    metrics_df.set_index('Time Step', inplace=True)

    return metrics_df

In [1]:
# test 1
# Number of runs
n_runs = 10

length = 24
hyperparams = best_hyperparameters[length]

# Initialize lists to store mean metrics and all metrics from each run
mean_metrics_rows = []
df_all_metrics_list = []

for run in range(1, n_runs + 1):
    print(f"\n--- Run {run} ---")
    
    # Optionally set a unique seed for each run to ensure variability
    #seed = run  
    # Train the model
    model, mse = train_model(hyperparams, data_dict, length, seed=None)

    X_train = data_dict[length]['X_train']
    y_train = data_dict[length]['y_train']
    X_val = data_dict[length]['X_val']
    y_val = data_dict[length]['y_val']
    X_test = data_dict[length]['X_test']
    y_test = data_dict[length]['y_test']

    y_pred_train, y_obs_train = make_prediction(model, X_train, y_train)
    y_pred_val, y_obs_val = make_prediction(model, X_val, y_val)
    y_pred_test, y_obs_test = make_prediction(model, X_test, y_test)

    df_train = evaluation(y_pred_train, y_obs_train).add_suffix('_train')
    df_val = evaluation(y_pred_val, y_obs_val).add_suffix('_val')
    df_test = evaluation(y_pred_test, y_obs_test).add_suffix('_test')

    df_all_metrics = pd.concat([df_train, df_val, df_test], axis=1)
    df_all_metrics.index.name = length
    
    # Append df_all_metrics to the list
    df_all_metrics_list.append(df_all_metrics)

    # Calculate mean for each metric
    mean_metrics = df_val.mean()
    mean_metrics_row = pd.DataFrame(mean_metrics).T
    mean_metrics_row['MSE_val(loss)'] = mse
    mean_metrics_row['input_len'] = length
    mean_metrics_row = mean_metrics_row[['input_len','MSE_val(loss)', 'MAE_val', 'RMSE_val', 'MAPE (%)_val']]
    
    # Append to the list
    mean_metrics_rows.append(mean_metrics_row)

NameError: name 'best_hyperparameters' is not defined

In [ ]:
# test 2
# Concatenate all df_all_metrics into a single DataFrame with a new level for runs
concatenated_all_metrics = pd.concat(df_all_metrics_list, keys=range(1, n_runs + 1), names=['Run', 'Time Step'])

# Calculate the mean across runs for each metric and time step
# This will group by 'Time Step' and calculate the mean of each metric across all runs
aggregated_all_metrics_mean = concatenated_all_metrics.groupby('Time Step').mean()
aggregated_all_metrics_mean.index.name = length

print("\n--- Aggregated Mean of All Metrics Across 10 Runs ---")
display(aggregated_all_metrics_mean)

# After all runs, create a DataFrame of mean metrics
mean_metrics_df = pd.concat(mean_metrics_rows, ignore_index=True)
# Calculate the mean of each metric across the 10 runs
final_mean_metrics = mean_metrics_df.mean()

# Create a DataFrame for the final mean metrics
final_mean_metrics_df = pd.DataFrame(final_mean_metrics).T

print("\n--- Final Mean Metrics Across 10 Runs ---")
final_mean_metrics_df

In [ ]:
mean_metrics_list=[]
for length in best_hyperparameters.keys():
    print(length)
    
    # Number of runs
    n_runs = 10
    
    # get the best hyperparameter of each length
    hyperparams = best_hyperparameters[length]
    # Initialize lists to store mean metrics and all metrics from each run
    mean_metrics_rows = []
    df_all_metrics_list = []

    for run in range(1, n_runs + 1):
        print(f"\n--- Run {run} ---")

        # Optionally set a unique seed for each run to ensure variability
        seed = run  
        # Train the model
        model, mse = train_model(hyperparams, data_dict, length, seed=seed)
        
        X_train = data_dict[length]['X_train']
        y_train = data_dict[length]['y_train']
        X_val = data_dict[length]['X_val']
        y_val = data_dict[length]['y_val']
        X_test = data_dict[length]['X_test']
        y_test = data_dict[length]['y_test']

        #get the true flow and predicted flow
        y_pred_train, y_obs_train = make_prediction(model, X_train, y_train)
        y_pred_val, y_obs_val = make_prediction(model, X_val, y_val)
        y_pred_test, y_obs_test = make_prediction(model, X_test, y_test)

        #calculate the evaluation metrics of each output step
        df_train = evaluation(y_pred_train, y_obs_train).add_suffix('_train')
        df_val = evaluation(y_pred_val, y_obs_val).add_suffix('_val')
        df_test = evaluation(y_pred_test, y_obs_test).add_suffix('_test')

        df_all_metrics = pd.concat([df_train, df_val, df_test], axis=1)
        df_all_metrics.index.name = length
        
        # Append df_all_metrics to the list
        df_all_metrics_list.append(df_all_metrics)

        # Calculate mean for all output step
        mean_metrics = df_val.mean()
        mean_metrics_row = pd.DataFrame(mean_metrics).T
        mean_metrics_row['MSE_val(loss)'] = mse
        mean_metrics_row['input_len'] = length
        mean_metrics_row = mean_metrics_row[['input_len','MSE_val(loss)', 'MAE_val', 'RMSE_val', 'MAPE (%)_val']]
        
        # Append to the list
        mean_metrics_rows.append(mean_metrics_row)
        
    # Concatenate all df_all_metrics into a single DataFrame with a new level for runs
    concatenated_all_metrics = pd.concat(df_all_metrics_list, keys=range(1, n_runs + 1), names=['Run', 'Time Step'])

    # Calculate the mean across runs for each metric and time step
    # This will group by 'Time Step' and calculate the mean of each metric across all runs
    aggregated_all_metrics_mean = concatenated_all_metrics.groupby('Time Step').mean()
    aggregated_all_metrics_mean.index.name = length

    print("\n--- Aggregated Mean of All Metrics Across 10 Runs ---")
    display(aggregated_all_metrics_mean)

    # After all runs, create a DataFrame of mean metrics
    mean_metrics_df = pd.concat(mean_metrics_rows, ignore_index=True)
    # Calculate the mean of each metric across the 10 runs
    final_mean_metrics = mean_metrics_df.mean()

    # Create a DataFrame for the final mean metrics
    final_mean_metrics_df = pd.DataFrame(final_mean_metrics).T
    
    mean_metrics_list.append(final_mean_metrics_df)

mean_metrics_df = pd.concat(mean_metrics_list).reset_index(drop=True)
print("\n--- Final Mean Metrics Across 10 Runs ---")
mean_metrics_df

24

--- Run 1 ---


E:\program\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 47: early stopping
Restoring model weights from the end of the best epoch: 27.
Validation loss: 0.00432

--- Run 2 ---


E:\program\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 44: early stopping
Restoring model weights from the end of the best epoch: 24.
Validation loss: 0.00423

--- Run 3 ---


...

E:\program\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 36: early stopping
Restoring model weights from the end of the best epoch: 16.
Validation loss: 0.00424

--- Run 6 ---


E:\program\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 54: early stopping
Restoring model weights from the end of the best epoch: 34.
Validation loss: 0.00426

--- Run 7 ---


E:\program\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [19]:
mean_metrics_df

,input_len,MSE_val(loss),MAE_val,RMSE_val,MAPE (%)_val
0,24.0,0.004265,26.205162,36.255125,43.886447
1,48.0,0.004287,26.229182,36.345324,42.311301
2,72.0,0.004252,26.128547,36.225445,42.704548
3,96.0,0.004268,26.183151,36.258214,43.057429
4,120.0,0.004109,25.769214,35.570711,43.494388
5,144.0,0.004119,25.819611,35.626515,43.322926
6,168.0,0.003862,24.786606,34.492432,41.654022
7,192.0,0.003810,24.557236,34.295043,38.974767
8,216.0,0.003502,24.161857,32.885453,40.072552
9,240.0,0.003759,25.139706,34.073613,41.733799


## 9. Store the results

In [31]:
# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

# Expand the hyperparameters dictionary into separate columns for clarity
hyperparams_df = results_df['Best_Hyperparameters'].apply(pd.Series)

# Combine the main dataframe with hyperparameters
final_results_df = pd.concat([results_df.drop('Best_Hyperparameters', axis=1), hyperparams_df], axis=1)

# Display the final dataframe
print("Final Results DataFrame:")
final_results_df

Final Results DataFrame:


,Input_Length,Best_MSE,Validation_MSE,Validation_MAE,Validation_MAPE,Validation_RMSE,filters,kernel_size,dropout,learning_rate,batch_size
0,192,0.003742,0.003742,0.044218,48.484668,0.061174,128.0,3.0,0.2,0.0001,32.0
1,216,0.003442,0.003442,0.043077,48.433814,0.058665,64.0,3.0,0.2,0.0001,32.0
2,240,0.003536,0.003536,0.043359,45.041577,0.059463,32.0,5.0,0.0,0.0001,32.0
3,264,0.003598,0.003598,0.043513,45.719254,0.059979,128.0,3.0,0.3,0.0005,32.0
4,288,0.003510,0.003510,0.042542,45.630643,0.059243,32.0,3.0,0.3,0.0001,32.0
5,312,0.003469,0.003469,0.043472,47.613100,0.058897,32.0,2.0,0.1,0.0005,128.0
6,336,0.003374,0.003374,0.042398,51.722801,0.058084,64.0,2.0,0.2,0.0001,32.0


## 5. Define the CNN model

In [17]:
def create_cnn_model(input_shape, n_outputs, dropout_rate=0.5, learning_rate=0.001, filters=64, kernel_size=3):
    model = keras.Sequential([
        layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=input_shape),
        layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'),
        layers.MaxPooling1D(pool_size=2),
        layers.Dropout(dropout_rate),
        layers.Flatten(),
        layers.Dense(50, activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Dense(n_outputs)
    ])
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        loss='mse',
        optimizer=optimizer,
        metrics=['mae']
    )
    return model

input_shape = (x_train_scaled.shape[1], x_train_scaled.shape[2])
n_outputs = y_train.shape[1]

## 6. Set Up Hyperparameter Grid

In [18]:
from itertools import product

# Define hyperparameter options
dropout_rates = [0.3, 0.5, 0.7]
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [32, 64, 128]
filters_list = [32, 64, 128]
kernel_sizes = [2, 3, 5]

# Create a list of all hyperparameter combinations
hyperparameter_combinations = list(product(dropout_rates, learning_rates, batch_sizes, filters_list, kernel_sizes))

print(f"Total combinations: {len(hyperparameter_combinations)}")

Total combinations: 243


## 7. Train the Model with Hyperparameter Tuning

In [20]:
best_val_mae = np.inf
best_hyperparams = None
best_model = None

for idx, (dropout_rate, learning_rate, batch_size, filters, kernel_size) in enumerate(hyperparameter_combinations):
    print(f"\nCombination {idx+1}/{len(hyperparameter_combinations)}")
    print(f"Training with dropout_rate={dropout_rate}, learning_rate={learning_rate}, batch_size={batch_size}, filters={filters}, kernel_size={kernel_size}")

    model = create_cnn_model(
        input_shape, n_outputs,
        dropout_rate=dropout_rate,
        learning_rate=learning_rate,
        filters=filters,
        kernel_size=kernel_size
    )

    # Initialize EarlyStopping inside the loop
    early_stopping = EarlyStopping(
        monitor='val_mae',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )

    history = model.fit(
        x_train_scaled, y_train_scaled,
        epochs=50,  # Set higher epochs due to Early Stopping
        batch_size=batch_size,
        validation_data=(x_val_scaled, y_val_scaled),
        callbacks=[early_stopping],
        verbose=0  # Change to 1 for detailed output
    )

    val_mae = min(history.history['val_mae'])
    print(f"Validation MAE: {val_mae:.4f}")

    if val_mae < best_val_mae:
        best_val_mae = val_mae
        best_model = model
        best_hyperparams = {
            'dropout_rate': dropout_rate,
            'learning_rate': learning_rate,
            'batch_size': batch_size,
            'filters': filters,
            'kernel_size': kernel_size
        }

print("\nBest Hyperparameters:")
for param, value in best_hyperparams.items():
    print(f"{param}: {value}")
print(f"Best Validation MAE: {best_val_mae:.4f}")



Combination 1/243
Training with dropout_rate=0.3, learning_rate=0.01, batch_size=32, filters=32, kernel_size=2
Epoch 34: early stopping
Restoring model weights from the end of the best epoch: 24.
Validation MAE: 0.0675

Combination 2/243
Training with dropout_rate=0.3, learning_rate=0.01, batch_size=32, filters=32, kernel_size=3
Epoch 34: early stopping
Restoring model weights from the end of the best epoch: 24.
Validation MAE: 0.0637

Combination 3/243
Training with dropout_rate=0.3, learning_rate=0.01, batch_size=32, filters=32, kernel_size=5
Epoch 20: early stopping
Restoring model weights from the end of the best epoch: 10.
Validation MAE: 0.0661

Combination 4/243
Training with dropout_rate=0.3, learning_rate=0.01, batch_size=32, filters=64, kernel_size=2
Epoch 26: early stopping
Restoring model weights from the end of the best epoch: 16.
Validation MAE: 0.0647

Combination 5/243
Training with dropout_rate=0.3, learning_rate=0.01, batch_size=32, filters=64, kernel_size=3
Epoch 19

Epoch 40: early stopping
Restoring model weights from the end of the best epoch: 30.
Validation MAE: 0.0605

Combination 39/243
Training with dropout_rate=0.3, learning_rate=0.001, batch_size=64, filters=32, kernel_size=5
Epoch 29: early stopping
Restoring model weights from the end of the best epoch: 19.
Validation MAE: 0.0613

Combination 40/243
Training with dropout_rate=0.3, learning_rate=0.001, batch_size=64, filters=64, kernel_size=2
Epoch 25: early stopping
Restoring model weights from the end of the best epoch: 15.
Validation MAE: 0.0618

Combination 41/243
Training with dropout_rate=0.3, learning_rate=0.001, batch_size=64, filters=64, kernel_size=3
Epoch 41: early stopping
Restoring model weights from the end of the best epoch: 31.
Validation MAE: 0.0599

Combination 42/243
Training with dropout_rate=0.3, learning_rate=0.001, batch_size=64, filters=64, kernel_size=5
Epoch 33: early stopping
Restoring model weights from the end of the best epoch: 23.
Validation MAE: 0.0603

Com

Restoring model weights from the end of the best epoch: 47.
Validation MAE: 0.0626

Combination 79/243
Training with dropout_rate=0.3, learning_rate=0.0001, batch_size=128, filters=128, kernel_size=2
Restoring model weights from the end of the best epoch: 49.
Validation MAE: 0.0637

Combination 80/243
Training with dropout_rate=0.3, learning_rate=0.0001, batch_size=128, filters=128, kernel_size=3
Restoring model weights from the end of the best epoch: 42.
Validation MAE: 0.0619

Combination 81/243
Training with dropout_rate=0.3, learning_rate=0.0001, batch_size=128, filters=128, kernel_size=5
Restoring model weights from the end of the best epoch: 47.
Validation MAE: 0.0612

Combination 82/243
Training with dropout_rate=0.5, learning_rate=0.01, batch_size=32, filters=32, kernel_size=2
Epoch 48: early stopping
Restoring model weights from the end of the best epoch: 38.
Validation MAE: 0.0721

Combination 83/243
Training with dropout_rate=0.5, learning_rate=0.01, batch_size=32, filters=3

Epoch 42: early stopping
Restoring model weights from the end of the best epoch: 32.
Validation MAE: 0.0612

Combination 117/243
Training with dropout_rate=0.5, learning_rate=0.001, batch_size=32, filters=128, kernel_size=5
Epoch 25: early stopping
Restoring model weights from the end of the best epoch: 15.
Validation MAE: 0.0622

Combination 118/243
Training with dropout_rate=0.5, learning_rate=0.001, batch_size=64, filters=32, kernel_size=2
Epoch 39: early stopping
Restoring model weights from the end of the best epoch: 29.
Validation MAE: 0.0655

Combination 119/243
Training with dropout_rate=0.5, learning_rate=0.001, batch_size=64, filters=32, kernel_size=3
Epoch 34: early stopping
Restoring model weights from the end of the best epoch: 24.
Validation MAE: 0.0631

Combination 120/243
Training with dropout_rate=0.5, learning_rate=0.001, batch_size=64, filters=32, kernel_size=5
Epoch 45: early stopping
Restoring model weights from the end of the best epoch: 35.
Validation MAE: 0.0650

Validation MAE: 0.0688

Combination 156/243
Training with dropout_rate=0.5, learning_rate=0.0001, batch_size=128, filters=32, kernel_size=5
Restoring model weights from the end of the best epoch: 45.
Validation MAE: 0.0715

Combination 157/243
Training with dropout_rate=0.5, learning_rate=0.0001, batch_size=128, filters=64, kernel_size=2
Restoring model weights from the end of the best epoch: 48.
Validation MAE: 0.0685

Combination 158/243
Training with dropout_rate=0.5, learning_rate=0.0001, batch_size=128, filters=64, kernel_size=3
Restoring model weights from the end of the best epoch: 50.
Validation MAE: 0.0656

Combination 159/243
Training with dropout_rate=0.5, learning_rate=0.0001, batch_size=128, filters=64, kernel_size=5
Restoring model weights from the end of the best epoch: 50.
Validation MAE: 0.0659

Combination 160/243
Training with dropout_rate=0.5, learning_rate=0.0001, batch_size=128, filters=128, kernel_size=2
Restoring model weights from the end of the best epoch: 47.

Validation MAE: 0.0721

Combination 194/243
Training with dropout_rate=0.7, learning_rate=0.001, batch_size=32, filters=64, kernel_size=3
Epoch 39: early stopping
Restoring model weights from the end of the best epoch: 29.
Validation MAE: 0.0682

Combination 195/243
Training with dropout_rate=0.7, learning_rate=0.001, batch_size=32, filters=64, kernel_size=5
Epoch 47: early stopping
Restoring model weights from the end of the best epoch: 37.
Validation MAE: 0.0682

Combination 196/243
Training with dropout_rate=0.7, learning_rate=0.001, batch_size=32, filters=128, kernel_size=2
Epoch 45: early stopping
Restoring model weights from the end of the best epoch: 35.
Validation MAE: 0.0682

Combination 197/243
Training with dropout_rate=0.7, learning_rate=0.001, batch_size=32, filters=128, kernel_size=3
Restoring model weights from the end of the best epoch: 41.
Validation MAE: 0.0663

Combination 198/243
Training with dropout_rate=0.7, learning_rate=0.001, batch_size=32, filters=128, kernel

Restoring model weights from the end of the best epoch: 49.
Validation MAE: 0.0676

Combination 234/243
Training with dropout_rate=0.7, learning_rate=0.0001, batch_size=64, filters=128, kernel_size=5
Restoring model weights from the end of the best epoch: 45.
Validation MAE: 0.0682

Combination 235/243
Training with dropout_rate=0.7, learning_rate=0.0001, batch_size=128, filters=32, kernel_size=2
Restoring model weights from the end of the best epoch: 49.
Validation MAE: 0.0858

Combination 236/243
Training with dropout_rate=0.7, learning_rate=0.0001, batch_size=128, filters=32, kernel_size=3
Restoring model weights from the end of the best epoch: 48.
Validation MAE: 0.0840

Combination 237/243
Training with dropout_rate=0.7, learning_rate=0.0001, batch_size=128, filters=32, kernel_size=5
Epoch 39: early stopping
Restoring model weights from the end of the best epoch: 29.
Validation MAE: 0.0873

Combination 238/243
Training with dropout_rate=0.7, learning_rate=0.0001, batch_size=128, f

Best Hyperparameters for full time:
dropout_rate: 0.3
learning_rate: 0.001
batch_size: 64
filters: 128
kernel_size: 5

Best Hyperparameters for after covid:
dropout_rate: 0.3
learning_rate: 0.0001
batch_size: 32
filters: 128
kernel_size: 5
Best Validation MAE: 0.0592

## 8. Make Predictions and Get Evaluation Metrics
I forget the store the results of best_model, but still have the results of best_hyperparameter

So I have to recreate the model using these hyperparameters and retrain it

#### 8.1 Recreate the model

In [ ]:
def create_best_cnn_model(input_shape, n_outputs):
    model = keras.Sequential([
        # First convolutional layer
        layers.Conv1D(filters=128, kernel_size=5, activation='relu', input_shape=input_shape),
        # Second convolutional layer
        layers.Conv1D(filters=128, kernel_size=5, activation='relu'),
        # Pooling layer
        layers.MaxPooling1D(pool_size=2),
        # Regularization
        layers.Dropout(0.3),
        # Flattening the output
        layers.Flatten(),
        # Fully connected layers
        layers.Dense(50, activation='relu'),
        layers.Dropout(0.3),
        # Output layer
        layers.Dense(n_outputs)
    ])
    # Compile the model
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(
        loss='mse',
        optimizer=optimizer,
        metrics=['mae']
    )
    return model

In [ ]:
# Recreate the model
input_shape = (x_train_scaled.shape[1], x_train_scaled.shape[2])
n_outputs = y_train.shape[1]
best_model = create_best_cnn_model(input_shape, n_outputs)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


#### 8.2 Retrain the model

In [ ]:
# Initialize EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_mae',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

In [ ]:
# Train the model
history = best_model.fit(
    x_train_scaled, y_train_scaled,
    epochs=50,  # You can adjust this as needed
    batch_size=64,
    validation_data=(x_val_scaled, y_val_scaled),
    callbacks=[early_stopping],
    verbose=1  # Set to 1 to see detailed training output
)

Epoch 1/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - loss: 0.0264 - mae: 0.1218 - val_loss: 0.0072 - val_mae: 0.0583
Epoch 2/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0117 - mae: 0.0809 - val_loss: 0.0071 - val_mae: 0.0575
Epoch 3/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0102 - mae: 0.0753 - val_loss: 0.0070 - val_mae: 0.0555
Epoch 4/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0096 - mae: 0.0728 - val_loss: 0.0070 - val_mae: 0.0537
Epoch 5/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0094 - mae: 0.0723 - val_loss: 0.0072 - val_mae: 0.0589
Epoch 6/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0092 - mae: 0.0713 - val_loss: 0.0068 - val_mae: 0.0551
Epoch 7/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0090 - mae: 0.0701 - val_loss: 0.0069 - val_mae: 0.0573
Epoch 8/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0087 - mae: 0.0692 - val_loss: 0.0071 - val_mae: 0.0587
Epoch 9/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - l

#### 8.3 Make Predictions and Get Evaluation Metrics

In [21]:
# Make predictions
y_pred_scaled = best_model.predict(x_test_scaled)

# Reshape for inverse scaling
y_pred_reshaped = y_pred_scaled.reshape(-1, 1)
y_test_reshaped = y_test_scaled.reshape(-1, 1)

# Inverse transform
y_pred_inverse = y_scaler.inverse_transform(y_pred_reshaped).reshape(n_test_samples, n_outputs)
y_test_inverse = y_scaler.inverse_transform(y_test_reshaped).reshape(n_test_samples, n_outputs)

y_test_flat = y_test_inverse.flatten()
y_pred_flat = y_pred_inverse.flatten()

66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [22]:
# Compute Metrics for Each Time Step

for i in range(n_outputs):
    y_true = y_test_inverse[:, i]
    y_pred = y_pred_inverse[:, i]

    # Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_true, y_pred)

    # Mean Squared Error (MSE)
    mse = mean_squared_error(y_true, y_pred)

    # Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)

    # Mean Absolute Percentage Error (MAPE)
    # Avoid division by zero by adding a small epsilon to y_test_flat if necessary
    epsilon = 1e-10
    y_true_safe = np.where(y_true == 0, epsilon, y_true)
    mape = np.mean(np.abs((y_true - y_pred) / y_true_safe)) * 100

    print(f"\nTime Step {i+1} Evaluation Metrics:")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"MAPE: {mape:.2f}%")



Time Step 1 Evaluation Metrics:
RMSE: 28.3201
MAE: 21.0287
MAPE: 20.43%

Time Step 2 Evaluation Metrics:
RMSE: 36.8282
MAE: 27.7824
MAPE: 26.41%

Time Step 3 Evaluation Metrics:
RMSE: 44.9135
MAE: 34.3549
MAPE: 31.76%

Time Step 4 Evaluation Metrics:
RMSE: 51.2590
MAE: 39.5928
MAPE: 36.60%

Time Step 5 Evaluation Metrics:
RMSE: 57.0935
MAE: 44.7165
MAPE: 41.28%

Time Step 6 Evaluation Metrics:
RMSE: 59.9910
MAE: 47.6431
MAPE: 44.87%


# Code to predict next 6 steps step-by-step

#### We will use
* the last 12 steps

* previous one week (24 steps)

* previous one month  (168 steps)

*  to forecast current (0 step)

## 1. Create input and output data

In [134]:
# Create input-output sequences with the provided function
X_train, y_train, X_train_df, y_train_df = create_multi_step_sequence(train_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=1)
X_valid, y_valid, X_valid_df, y_valid_df = create_multi_step_sequence(valid_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=1)
X_test, y_test, X_test_df, y_test_df = create_multi_step_sequence(test_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=1)

In [118]:
X_train.shape, X_train, y_train.shape, y_train

((19570, 14, 1),
 array([[[173.],
         [168.],
         [155.],
         ...,
         [ 27.],
         [ 21.],
         [ 15.]],
 
        [[168.],
         [155.],
         [186.],
         ...,
         [  9.],
         [  8.],
         [  9.]],
 
        [[155.],
         [186.],
         [333.],
         ...,
         [ 10.],
         [ 10.],
         [  9.]],
 
        ...,
 
        [[141.],
         [142.],
         [107.],
         ...,
         [160.],
         [129.],
         [131.]],
 
        [[142.],
         [107.],
         [128.],
         ...,
         [ 94.],
         [ 87.],
         [ 77.]],
 
        [[107.],
         [128.],
         [150.],
         ...,
         [ 80.],
         [ 63.],
         [ 35.]]]),
 (19570, 1),
 array([[ 9.],
        [10.],
        [ 8.],
        ...,
        [94.],
        [80.],
        [63.]]))

## 2. Normalise the data after split (step-by-step)

Normalise X

In [135]:
# Separate scalers for inputs and outputs
x_scaler = MinMaxScaler(feature_range=(0, 1))
y_scaler = MinMaxScaler(feature_range=(0, 1))

# Reshape x_train to 2D for scaling
n_samples, n_timesteps, n_features = X_train.shape
x_train_reshaped = X_train.reshape(-1, n_features)  # Shape: (n_samples * n_timesteps, n_features)
# Fit the scaler on the training data
x_scaler.fit(x_train_reshaped)
# Transform the training data
x_train_scaled = x_scaler.transform(x_train_reshaped)
# Reshape back to original shape
x_train_scaled = x_train_scaled.reshape(n_samples, n_timesteps, n_features)

# x_val
n_val_samples = X_valid.shape[0]
x_val_reshaped = X_valid.reshape(-1, n_features)
x_val_scaled = x_scaler.transform(x_val_reshaped)
x_val_scaled = x_val_scaled.reshape(n_val_samples, n_timesteps, n_features)

# x_test
n_test_samples = X_test.shape[0]
x_test_reshaped = X_test.reshape(-1, n_features)
x_test_scaled = x_scaler.transform(x_test_reshaped)
x_test_scaled = x_test_scaled.reshape(n_test_samples, n_timesteps, n_features)

Normalise y

In [136]:
# Reshape y_train to 2D for scaling
y_train_reshaped = y_train.reshape(-1, 1)  # Shape: (n_samples * n_outputs, 1)
# Fit the scaler on the training data
y_scaler.fit(y_train_reshaped)
# Transform the training data
y_train_scaled = y_scaler.transform(y_train_reshaped)
# Reshape back to original shape
y_train_scaled = y_train_scaled.reshape(n_samples, y_train.shape[1])

# y_val
y_val_reshaped = y_valid.reshape(-1, 1)
y_val_scaled = y_scaler.transform(y_val_reshaped)
y_val_scaled = y_val_scaled.reshape(n_val_samples, y_valid.shape[1])

# y_test
y_test_reshaped = y_test.reshape(-1, 1)
y_test_scaled = y_scaler.transform(y_test_reshaped)
y_test_scaled = y_test_scaled.reshape(n_test_samples, y_test.shape[1])

## 3. Build the CNN model (step-by-step)

In [26]:
def create_cnn_model_recursive(input_shape,
                               dropout_rate=0.5,
                               learning_rate=0.001,
                               filters=64,
                               kernel_size=3):
    model = keras.Sequential([
        layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=input_shape),
        layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'),
        layers.MaxPooling1D(pool_size=2),
        layers.Dropout(dropout_rate),
        layers.Flatten(),
        layers.Dense(50, activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Dense(1)  # Output layer for one-step prediction
    ])
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    return model

## 4. Define Separate Hyperparameter Grids

In [27]:
from itertools import product

In [28]:
dropout_rates = [0, 0.3, 0.5]
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [32, 64, 128]
filters_list = [32, 64, 128]
kernel_sizes = [2, 3, 5]

# Create a list of all hyperparameter combinations
hyperparameter_combinations = list(product(dropout_rates,
                                           learning_rates, batch_sizes,
                                           filters_list,
                                           kernel_sizes))


## 5. Tuning hyperparameter

In [29]:
best_val_mae = np.inf
best_hyperparams = None
best_model = None

for idx, (dropout_rate, learning_rate, batch_size,
          filters, kernel_size) in enumerate(hyperparameter_combinations):
    print(f"\nCombination {idx+1}/{len(hyperparameter_combinations)}")
    print(f"Training with dropout_rate={dropout_rate}, "
          f"learning_rate={learning_rate}, batch_size={batch_size}, "
          f"filters={filters}, "
          f"kernel_size={kernel_size}")

    model = create_cnn_model_recursive(
        input_shape=(n_timesteps, 1),
        dropout_rate=dropout_rate,
        learning_rate=learning_rate,
        filters=filters,
        kernel_size=kernel_size
    )

    # Initialize EarlyStopping inside the loop
    early_stopping = EarlyStopping(
        monitor='val_mae',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )

    # Train the model
    history = model.fit(
        x_train_scaled, y_train_scaled,
        epochs=50,
        batch_size=batch_size,
        validation_data=(x_val_scaled, y_val_scaled),
        callbacks=[early_stopping],
        verbose=0  # Set to 1 to see detailed training output
    )

    # Get the best validation MAE from this training run
    val_mae = min(history.history['val_mae'])
    print(f"Validation MAE: {val_mae:.4f}")

    # Update best model if current one is better
    if val_mae < best_val_mae:
        best_val_mae = val_mae
        best_model = model
        best_hyperparams = {
            'dropout_rate': dropout_rate,
            'learning_rate': learning_rate,
            'batch_size': batch_size,
            'filters': filters,
            'kernel_size': kernel_size
        }

print("\nBest Hyperparameters:")
for param, value in best_hyperparams.items():
    print(f"{param}: {value}")
print(f"Best Validation MAE: {best_val_mae:.4f}")


Combination 1/243
Training with dropout_rate=0, learning_rate=0.01, batch_size=32, filters=32, kernel_size=2


E:\program\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 41: early stopping
Restoring model weights from the end of the best epoch: 31.
Validation MAE: 0.0463

Combination 2/243
Training with dropout_rate=0, learning_rate=0.01, batch_size=32, filters=32, kernel_size=3
Epoch 18: early stopping
Restoring model weights from the end of the best epoch: 8.
Validation MAE: 0.0468

Combination 3/243
Training with dropout_rate=0, learning_rate=0.01, batch_size=32, filters=32, kernel_size=5
Epoch 22: early stopping
Restoring model weights from the end of the best epoch: 12.
Validation MAE: 0.0468

Combination 4/243
Training with dropout_rate=0, learning_rate=0.01, batch_size=32, filters=64, kernel_size=2
Epoch 26: early stopping
Restoring model weights from the end of the best epoch: 16.
Validation MAE: 0.0471

Combination 5/243
Training with dropout_rate=0, learning_rate=0.01, batch_size=32, filters=64, kernel_size=3
Epoch 23: early stopping
Restoring model weights from the end of the best epoch: 13.
Validation MAE: 0.0476

Combination 6/243
Tr

Validation MAE: 0.0452

Combination 39/243
Training with dropout_rate=0, learning_rate=0.001, batch_size=64, filters=32, kernel_size=5
Epoch 33: early stopping
Restoring model weights from the end of the best epoch: 23.
Validation MAE: 0.0453

Combination 40/243
Training with dropout_rate=0, learning_rate=0.001, batch_size=64, filters=64, kernel_size=2
Epoch 21: early stopping
Restoring model weights from the end of the best epoch: 11.
Validation MAE: 0.0454

Combination 41/243
Training with dropout_rate=0, learning_rate=0.001, batch_size=64, filters=64, kernel_size=3
Epoch 22: early stopping
Restoring model weights from the end of the best epoch: 12.
Validation MAE: 0.0449

Combination 42/243
Training with dropout_rate=0, learning_rate=0.001, batch_size=64, filters=64, kernel_size=5
Epoch 18: early stopping
Restoring model weights from the end of the best epoch: 8.
Validation MAE: 0.0460

Combination 43/243
Training with dropout_rate=0, learning_rate=0.001, batch_size=64, filters=128,

Restoring model weights from the end of the best epoch: 45.
Validation MAE: 0.0450

Combination 78/243
Training with dropout_rate=0, learning_rate=0.0001, batch_size=128, filters=64, kernel_size=5
Restoring model weights from the end of the best epoch: 44.
Validation MAE: 0.0450

Combination 79/243
Training with dropout_rate=0, learning_rate=0.0001, batch_size=128, filters=128, kernel_size=2
Epoch 35: early stopping
Restoring model weights from the end of the best epoch: 25.
Validation MAE: 0.0452

Combination 80/243
Training with dropout_rate=0, learning_rate=0.0001, batch_size=128, filters=128, kernel_size=3
Restoring model weights from the end of the best epoch: 46.
Validation MAE: 0.0449

Combination 81/243
Training with dropout_rate=0, learning_rate=0.0001, batch_size=128, filters=128, kernel_size=5
Epoch 48: early stopping
Restoring model weights from the end of the best epoch: 38.
Validation MAE: 0.0449

Combination 82/243
Training with dropout_rate=0.3, learning_rate=0.01, batc

Validation MAE: 0.0460

Combination 115/243
Training with dropout_rate=0.3, learning_rate=0.001, batch_size=32, filters=128, kernel_size=2
Epoch 18: early stopping
Restoring model weights from the end of the best epoch: 8.
Validation MAE: 0.0469

Combination 116/243
Training with dropout_rate=0.3, learning_rate=0.001, batch_size=32, filters=128, kernel_size=3
Epoch 13: early stopping
Restoring model weights from the end of the best epoch: 3.
Validation MAE: 0.0460

Combination 117/243
Training with dropout_rate=0.3, learning_rate=0.001, batch_size=32, filters=128, kernel_size=5
Epoch 35: early stopping
Restoring model weights from the end of the best epoch: 25.
Validation MAE: 0.0457

Combination 118/243
Training with dropout_rate=0.3, learning_rate=0.001, batch_size=64, filters=32, kernel_size=2
Epoch 39: early stopping
Restoring model weights from the end of the best epoch: 29.
Validation MAE: 0.0458

Combination 119/243
Training with dropout_rate=0.3, learning_rate=0.001, batch_size

Epoch 36: early stopping
Restoring model weights from the end of the best epoch: 26.
Validation MAE: 0.0457

Combination 153/243
Training with dropout_rate=0.3, learning_rate=0.0001, batch_size=64, filters=128, kernel_size=5
Restoring model weights from the end of the best epoch: 49.
Validation MAE: 0.0450

Combination 154/243
Training with dropout_rate=0.3, learning_rate=0.0001, batch_size=128, filters=32, kernel_size=2
Restoring model weights from the end of the best epoch: 49.
Validation MAE: 0.0482

Combination 155/243
Training with dropout_rate=0.3, learning_rate=0.0001, batch_size=128, filters=32, kernel_size=3
Restoring model weights from the end of the best epoch: 50.
Validation MAE: 0.0473

Combination 156/243
Training with dropout_rate=0.3, learning_rate=0.0001, batch_size=128, filters=32, kernel_size=5
Restoring model weights from the end of the best epoch: 45.
Validation MAE: 0.0473

Combination 157/243
Training with dropout_rate=0.3, learning_rate=0.0001, batch_size=128, f

Epoch 22: early stopping
Restoring model weights from the end of the best epoch: 12.
Validation MAE: 0.0483

Combination 191/243
Training with dropout_rate=0.5, learning_rate=0.001, batch_size=32, filters=32, kernel_size=3
Epoch 25: early stopping
Restoring model weights from the end of the best epoch: 15.
Validation MAE: 0.0484

Combination 192/243
Training with dropout_rate=0.5, learning_rate=0.001, batch_size=32, filters=32, kernel_size=5
Epoch 33: early stopping
Restoring model weights from the end of the best epoch: 23.
Validation MAE: 0.0483

Combination 193/243
Training with dropout_rate=0.5, learning_rate=0.001, batch_size=32, filters=64, kernel_size=2
Epoch 28: early stopping
Restoring model weights from the end of the best epoch: 18.
Validation MAE: 0.0471

Combination 194/243
Training with dropout_rate=0.5, learning_rate=0.001, batch_size=32, filters=64, kernel_size=3
Epoch 27: early stopping
Restoring model weights from the end of the best epoch: 17.
Validation MAE: 0.0468


Restoring model weights from the end of the best epoch: 42.
Validation MAE: 0.0493

Combination 229/243
Training with dropout_rate=0.5, learning_rate=0.0001, batch_size=64, filters=64, kernel_size=2
Epoch 42: early stopping
Restoring model weights from the end of the best epoch: 32.
Validation MAE: 0.0478

Combination 230/243
Training with dropout_rate=0.5, learning_rate=0.0001, batch_size=64, filters=64, kernel_size=3
Restoring model weights from the end of the best epoch: 42.
Validation MAE: 0.0470

Combination 231/243
Training with dropout_rate=0.5, learning_rate=0.0001, batch_size=64, filters=64, kernel_size=5
Epoch 44: early stopping
Restoring model weights from the end of the best epoch: 34.
Validation MAE: 0.0474

Combination 232/243
Training with dropout_rate=0.5, learning_rate=0.0001, batch_size=64, filters=128, kernel_size=2
Restoring model weights from the end of the best epoch: 49.
Validation MAE: 0.0468

Combination 233/243
Training with dropout_rate=0.5, learning_rate=0.0

Best hyperparameter for full time:
'dropout_rate': 0,
 'learning_rate': 0.001,
 'batch_size': 32,
 'filters': 128,
 'kernel_size': 3
 
Best hyperparameter for after covid:
dropout_rate: 0
learning_rate: 0.0001
batch_size: 64
filters: 128
kernel_size: 3

#### 5.1 Recreate the model

In [138]:
def create_best_cnn_model(input_shape, n_outputs):
    model = keras.Sequential([
        # First convolutional layer
        layers.Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=input_shape),
        # Second convolutional layer
        layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
        # Pooling layer
        layers.MaxPooling1D(pool_size=2),
        # Regularization
        layers.Dropout(0),
        # Flattening the output
        layers.Flatten(),
        # Fully connected layers
        layers.Dense(50, activation='relu'),
        layers.Dropout(0),
        # Output layer
        layers.Dense(n_outputs)
    ])
    # Compile the model
    optimizer = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(
        loss='mse',
        optimizer=optimizer,
        metrics=['mae']
    )
    return model

In [139]:
# Recreate the model
input_shape = (x_train_scaled.shape[1], x_train_scaled.shape[2])
n_outputs = y_train.shape[1]
best_model = create_best_cnn_model(input_shape, n_outputs)

E:\program\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [143]:
input_shape,n_outputs,x_test_scaled.shape

((14, 1), 1, (2141, 14, 1))

#### 5.2 Retrain the model

In [141]:
# Initialize EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_mae',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

In [142]:
# Train the model
history = best_model.fit(
    x_train_scaled, y_train_scaled,
    epochs=50,  # You can adjust this as needed
    batch_size=64,
    validation_data=(x_val_scaled, y_val_scaled),
    callbacks=[early_stopping],
    verbose=1  # Set to 1 to see detailed training output
)

Epoch 1/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0245 - mae: 0.1161 - val_loss: 0.0057 - val_mae: 0.0513
Epoch 2/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0048 - mae: 0.0457 - val_loss: 0.0055 - val_mae: 0.0491
Epoch 3/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0044 - mae: 0.0441 - val_loss: 0.0050 - val_mae: 0.0475
Epoch 4/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0042 - mae: 0.0424 - val_loss: 0.0049 - val_mae: 0.0467
Epoch 5/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0038 - mae: 0.0412 - val_loss: 0.0047 - val_mae: 0.0467
Epoch 6/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0042 - mae: 0.0423 - val_loss: 0.0059 - val_mae: 0.0512
Epoch 7/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0042 - mae: 0.0426 - val_loss: 0.0048 - val_mae: 0.0459
Epoch 8/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0040 - mae: 0.0415 - val_loss: 0.0047 - val_mae: 0.0462
Epoch 9/50
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - lo

## 6. Make prediction (step-by-step)

In [147]:
def recursive_forecast(model, x_test_scaled, start_index, n_steps, x_scaler, y_scaler):
    """
    Perform recursive forecasting using the trained model.

    Parameters:
    - model: Trained model
    - x_test_scaled: The scaled test input data (shape: n_samples, n_timesteps, n_features)
    - start_index: The starting index in the test data
    - n_steps: Number of future steps to predict
    - x_scaler: Scaler used for input features
    - y_scaler: Scaler used for target variable

    Returns:
    - predictions: List of predicted values (in original scale)
    """
    predictions = []
    scaled_predictions = []
    current_input = x_test_scaled[start_index].copy()  # Shape: (n_timesteps, n_features)

    for step in range(n_steps):
        # Reshape to (1, n_timesteps, n_features) for prediction
        input_seq = current_input.reshape((1, current_input.shape[0], current_input.shape[1]))

        # Predict the next time step (scaled)
        yhat_scaled = model.predict(input_seq, verbose=0)  # Shape: (1, 1)

        # Inverse transform the prediction to original scale
        yhat = y_scaler.inverse_transform(yhat_scaled)[0, 0]
        
        # Transform the prediction back to input feature scale for lag features
        yhat_for_input = x_scaler.transform(yhat.reshape(-1, 1))[0, 0]

        # Append predictions
        predictions.append(yhat)
        scaled_predictions.append(yhat_for_input)

        # Move to the next time step in x_test_scaled
        next_index = start_index + step + 1
        if next_index < len(x_test_scaled):
            # Use features from the next time step
            next_input = x_test_scaled[next_index].copy()
        else:
            # Reached the end of x_test_scaled
            break

        # Update lag features with available scaled predictions
        for lag in range(1, min(step + 1, 6) + 1):
            feature_index = 12 - lag  # lag1 is at index 11
            next_input[feature_index, 0] = scaled_predictions[-lag]

        # Keep lag24 and lag168 as they are, or update if necessary

        # Set current_input for next iteration
        current_input = next_input

    return predictions

In [148]:
# Number of steps to predict
n_steps = 6

# Initialize lists to store predictions and actual values
all_predictions = []
all_actuals = []

# Ensure we have enough data for recursive predictions
n_test_samples = x_test_scaled.shape[0]

for i in range(n_test_samples - n_steps):
    # Perform recursive forecasting
    predictions = recursive_forecast(
        model=best_model,
        x_test_scaled=x_test_scaled,
        start_index=i,
        n_steps=n_steps,
        x_scaler=x_scaler,
        y_scaler=y_scaler
    )

    # Get the actual future values (in original scale)
    actual_values = y_test[i+1:i + len(predictions) + 1].flatten()

    # Store the predictions and actual values
    all_predictions.append(predictions)
    all_actuals.append(actual_values)

# Convert lists to numpy arrays
all_predictions = np.array(all_predictions)
all_actuals = np.array(all_actuals)

In [149]:
epsilon = 1e-10  # For MAPE calculation to avoid division by zero

# Loop over each time step
for i in range(n_steps):
    y_true = all_actuals[:, i]
    y_pred = all_predictions[:, i]

    # Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_true, y_pred)

    # Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    # Mean Absolute Percentage Error (MAPE)
    y_true_safe = np.where(y_true == 0, epsilon, y_true)
    mape = np.mean(np.abs((y_true - y_pred) / y_true_safe)) * 100

    print(f"\nTime Step {i+1} Evaluation Metrics:")
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAPE: {mape:.2f}%")



Time Step 1 Evaluation Metrics:
MAE: 26.1654
RMSE: 34.5064
MAPE: 24.57%

Time Step 2 Evaluation Metrics:
MAE: 29.2920
RMSE: 38.6517
MAPE: 27.01%

Time Step 3 Evaluation Metrics:
MAE: 31.9744
RMSE: 42.0223
MAPE: 29.37%

Time Step 4 Evaluation Metrics:
MAE: 33.7413
RMSE: 44.1884
MAPE: 30.70%

Time Step 5 Evaluation Metrics:
MAE: 35.4154
RMSE: 46.1681
MAPE: 31.94%

Time Step 6 Evaluation Metrics:
MAE: 36.4250
RMSE: 47.3673
MAPE: 32.52%


In [ ]:
step-by-step after covid